Kod Podstawowy. Potrzebny do pracy ze sparkiem w colabie

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

findspark.init("spark-2.4.5-bin-hadoop2.7")
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName('abc').getOrCreate()

     |████████████████████████████████| 218.4MB 61kB/s 
     |████████████████████████████████| 204kB 49.4MB/s 


# Podstawy RDD

tworzenie RDD

In [0]:
rng = range(1,1000000)
nums = sc.parallelize(rng) # tworzy RDD ze zmiennej rng (przerzucenie zmiennej z pythona do sparka)
text_file = sc.textFile('path_to_file') # wczytywanie pliku tekstowego do RDD
nums.take(10) # zwraca x pierwszych elementow z RDD
nums.collect() # zbieranie danych z rdd do pythona
nums.count() # liczy ilosc elementow w rdd

Dodawanie i usuwanie zmiennych z pamieci podrecznej

In [0]:
nums_filtered.cache() # ładowanie zmiennej do pamięci podręcznej (RAM)
nums_filtered.unpersist() # usuwanie zmiennej z pamięci podręcznej (RAM)

# Transformacje i Akcje

## transformacja: leniwa operacja na RDD. Zwraca kolejny RDD. (map, filter, groupBy)

`rdd.map(func)` zwraca nowy RDD poprzez zaaplikowanie funkcji na kazdym elemencie

`dd.filter(func)`zwraca RDD zlożony z elementów, w których funkcja zwróciła True.

`rdd.flatMap(func)` podobne do map, ale ze spłaszczonym wymiarem wejściowym. (technicznie: może zwrócić dowolną liczbę elementów).

`rdd.mapPartitions(func)` podpbne do map, ale uruchamiane osobno na każdej partycji

`rdd.mapPartitionsWithIndex(func)` podobne do mapPartitions, ale pozwala na indentyfikację numery partycji.

`rdd.sample(withReplacement, fraction, seed)` losuje część obserwacji z danych (należy podać ułamek)

`rdd.union(otherDataset)` łączy RDD

`rdd.intersection(otherDataset)` zwraca przecięcie dwóch RDD

`rdd.distinct()` zwraca unikalne elementy w RDD

`rdd.groupByKey()` grupuje RDD po kluczu

`rd.reduceByKey(func)` redukuje po kluczu, podczas redukcji wykonuje dzialanie opisane w argumencie func

`rdd.sotrByKey()` sortuje

## akcje: operacja na rdd, która coś robi i zwraca zawartość RDD. (count, collect, save)

`rdd.reduce(func)` agregacja elementów za pomocą fukcji przyjmującej 2 elementy i zwracającej 1

`rdd.collect()` pobiera dane na sterownik klastra

`rdd.count()` przelicza elementy

`rdd.first()` zwraca pierwszy element

`rdd.take(n)` zwraca *n* pierwszych elementów

`rdd.takeSample(withReplacement, num)` zwraca tablice z losową próbką

`rdd.takeOrdered(n)` zwraca pierwsze n elementów z zachowaniem kolejności

`rdd.saveAsTextFile(path)` zapis do pliku tekstowego (lokalnie lub HDFS)

`rdd.countByKey()` liczy po kluczu

`rdd.foreach(func)` wykonanie funkcji na każdym elemencie zbioru danych

Funkcja `.map()` i `.flatMap()` przykłady działania

In [0]:
example_list = [1,2,3]

sc.parallelize(example_list).map(lambda x: [x,x,x]).collect()

[[1, 1, 1], [2, 2, 2], [3, 3, 3]]

In [0]:
sc.parallelize(example_list).flatMap(lambda x: [x,x,x]).collect() # spłaszcza o JEDEN wymiar

[1, 1, 1, 2, 2, 2, 3, 3, 3]

In [0]:
sc.parallelize(example_list).flatMap(lambda x: [[x,x,x]]).collect() # spłaszcza o JEDEN wymiar

[[1, 1, 1], [2, 2, 2], [3, 3, 3]]

Funkcja .mapParitions()

In [0]:
one_to_nine = [1,2,3,4,5,6,7,8,9]

def f(iterator):
  yield sum(iterator)

In [0]:
parallel = sc.parallelize(one_to_nine, 2)
parallel.mapPartitions(f).collect() # zwroci sume dla poszczegolnych partycji. 

[10, 35]

In [0]:
parallel = sc.parallelize(one_to_nine, 10)
parallel.mapPartitions(f).collect() # zwroci sume dla poszczegolnych partycji. 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Funkcja .mapPartitionsWithIndex()

In [0]:
def show(index, iterator):
  yield 'index: {}, values {}'.format(index, list(iterator))

In [0]:
parallel = sc.parallelize(one_to_nine, 3)
parallel.mapPartitionsWithIndex(show).collect()  # wykonuje dzialanie i zwraca indeks partycji

['index: 0, values [1, 2, 3]',
 'index: 1, values [4, 5, 6]',
 'index: 2, values [7, 8, 9]']

funkcja .sample()

In [0]:
parallel = sc.parallelize(one_to_nine, 2)
parallel.sample(True, .2).take(5)

[3, 3, 5]

funkcja .intersection()

In [0]:
one = sc.parallelize(range(1,10))
two = sc.parallelize(range(5,15))
one.intersection(two).collect() # zwraca czesc wspolna

[8, 5, 9, 6, 7]

funkcja .union() i .distinct()

In [0]:
parallel_1= sc.parallelize(range(1,9))
parallel_2= sc.parallelize(range(5,15))
parallel_1.union(parallel_2).collect() # łączy oba RDD

[1, 2, 3, 4, 5, 6, 7, 8, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [0]:
parallel_1.union(parallel_2).distinct().collect() # łączy oba rdd i usuwa duplikaty

[4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11]

Funkcja .aggregateByKey()

In [0]:
babies = sc.textFile('baby.csv') # wczytuje plik tekstowy .csv
rows = babies.map(lambda x: x.split(',')) # rozdzielam dane przecinkiem

In [17]:
(rows
 .filter(lambda x: 'Year' not in x)
 .map(lambda x: (x[1], int(x[4])))
 .aggregateByKey(0, lambda x,y: x+y, lambda x,y: x+y) # praktycznie to samo co reduce by key, bardziej niskopoziomowe.
 .sortBy(lambda x: x[1], False)
 ).take(10)

[('JAYDEN', 487),
 ('ETHAN', 412),
 ('DANIEL', 391),
 ('JACOB', 388),
 ('DAVID', 272),
 ('MOSHE', 219),
 ('SOPHIA', 204),
 ('ESTHER', 196),
 ('ISABELLA', 192),
 ('AIDEN', 182)]

funkcja .join() .leftOuterJoin()

In [0]:
names1 = sc.parallelize(('adam', 'bart', 'james')).map(lambda x: (x, 1))
names2 = sc.parallelize(('james', 'elon', 'steve')).map(lambda x: (x, 2))

In [11]:
names1.join(names2).collect()

[('james', (1, 2))]

In [12]:
names1.leftOuterJoin(names2).collect()

[('james', (1, 2)), ('bart', (1, None)), ('adam', (1, None))]

Iloczyn kartezjeński

In [14]:
one = sc.parallelize(range(1,10))
two = sc.parallelize(range(10,25))

cartesian_rdd = one.cartesian(two)
cartesian_rdd.take(10)

[(1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (2, 10),
 (3, 10),
 (2, 11)]

funkcja .coalesce() i .repartition()

In [15]:
one.coalesce(1).collect() # zweza ilosc partycji do podanej w argumencie, wazne podczas zapisu danych do pliku

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [0]:
one.repartition(2).collect() # zmienia ilosc partycji do podanej w argumencie

Połączone transformacje na RDD

In [0]:
(rows
 .filter(lambda x: 'Count' not in x) # usuwam naglowek
 .map(lambda x: (x[1], x[2])) # wybieram 2 i 3 kolumne z pliku csv
 .groupByKey() # grupuje po kluczu
 .map(lambda x: {x[0]: list(x[1])}) # tworze slownik gdzie klucz to x[0] a wartpsc dla klucza to rozpakowany iterator x[1]
 ).collect() # zbieram wynik leniwych transfromacji w nawiasie

[{'JAYDEN': ['KINGS', 'QUEENS']},
 {'ETHAN': ['QUEENS', 'KINGS']},
 {'SOPHIA': ['QUEENS']},
 {'DANIEL': ['QUEENS', 'KINGS']},
 {'ESTHER': ['KINGS']},
 {'ISABELLA': ['QUEENS']},
 {'AIDEN': ['KINGS']},
 {'NOAH': ['KINGS']},
 {'JOSEPH': ['KINGS']},
 {'DAVID': ['KINGS']},
 {'MOSHE': ['KINGS']},
 {'JACOB': ['KINGS', 'QUEENS']},
 {'LEAH': ['KINGS']},
 {'MATTHEW': ['QUEENS']}]

In [0]:
(rows
 .filter(lambda x: 'Count' not in x) #usuwam naglowek
 .map(lambda x: (x[1], int(x[4])))  #wybieram 2 i 5 kolumne, plus zamieniam  wartosci w 5 kolumnie na int
 .reduceByKey(lambda x,y: x+y) # dla dwoch wartosci z danego klucza wykonaj nastepujaca operacje
 ).collect()

[('DAVID', 272),
 ('JAYDEN', 268),
 ('JAYDEN', 219),
 ('MOSHE', 219),
 ('ETHAN', 216),
 ('SOPHIA', 204),
 ('DANIEL', 203),
 ('JACOB', 202),
 ('ESTHER', 196),
 ('ETHAN', 196),
 ('ISABELLA', 192),
 ('DANIEL', 188),
 ('JACOB', 186),
 ('AIDEN', 182),
 ('LEAH', 179),
 ('NOAH', 168),
 ('JOSEPH', 166),
 ('MATTHEW', 161)]

Broadcasting

In [0]:
sample_dict = {
    'a': 1, 
    'b': 2,
    'c': 3,
    'd': 4
    } # slownik ktory jest dostepny tylko dla komputera na ktorym pracuje

In [0]:
sample_dict_broadcasted = sc.broadcast(sample_dict) # udostepniam zmienna sample_dict dla wszystkich komputerow w klastrze

In [22]:
sample_dict_broadcasted.value # zeby odwolac sie do wartosci przechowywanej w zmiennej sample_dict_broadcasted, trzeba wywolac atrybut value

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

Akumulatory

In [0]:
accum = sc.accumulator(0) # tworze akumulator

In [0]:
sc.parallelize([1,2,3,4]).foreach(lambda x: accum.add(x)) # dodaje wartosci do akumulatora

In [26]:
accum.value # odwoluje sie dow artosci przechowywanej w akumulatorze

10